# High-ρ Model Hunt: Finding Other Dampening Models

**Paper #3 Experiment:** H25 Cross-Architecture Validation

**Hypothesis:** Models with ρ = n_heads / d_head > 0.2 should show DAMPENING (G < 1.0)

**Candidates to Test:**
- OPT family (Meta)
- BLOOM family (BigScience)
- Falcon family (TII)
- GPT-Neo family (EleutherAI)
- StableLM family (Stability AI)

**Goal:** Find at least ONE non-Pythia model that shows dampening due to high ρ

In [ ]:
# Install dependencies
!pip install transformers torch matplotlib numpy --quiet

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer
import json
from datetime import datetime
import warnings
import gc
warnings.filterwarnings('ignore')

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"GPU: {gpu_name}")
    print(f"GPU memory: {gpu_mem:.1f} GB")

In [ ]:
# Candidate models to analyze
# We'll first check their configs to compute ρ

CANDIDATE_MODELS = [
    # OPT Family
    'facebook/opt-125m',
    'facebook/opt-350m',
    'facebook/opt-1.3b',
    'facebook/opt-2.7b',
    'facebook/opt-6.7b',
    
    # BLOOM Family
    'bigscience/bloom-560m',
    'bigscience/bloom-1b1',
    'bigscience/bloom-1b7',
    'bigscience/bloom-3b',
    
    # Falcon Family
    'tiiuae/falcon-7b',
    
    # GPT-Neo Family
    'EleutherAI/gpt-neo-125M',
    'EleutherAI/gpt-neo-1.3B',
    'EleutherAI/gpt-neo-2.7B',
    
    # StableLM
    'stabilityai/stablelm-base-alpha-3b',
    
    # Reference models (known values)
    'EleutherAI/pythia-6.9b',  # ρ = 0.25, G ≈ 0.80
    'EleutherAI/gpt-j-6B',      # ρ = 0.0625, G ≈ 1.065
]

print(f"Candidate models: {len(CANDIDATE_MODELS)}")

In [ ]:
def get_model_rho(model_name):
    """Get architecture details and compute ρ from config."""
    try:
        config = AutoConfig.from_pretrained(model_name, trust_remote_code=True)
        
        # Get number of heads
        n_heads = getattr(config, 'num_attention_heads', None) or \
                  getattr(config, 'n_head', None) or \
                  getattr(config, 'num_heads', None)
        
        # Get hidden size
        d_model = getattr(config, 'hidden_size', None) or \
                  getattr(config, 'n_embd', None) or \
                  getattr(config, 'd_model', None)
        
        # Get number of layers
        n_layers = getattr(config, 'num_hidden_layers', None) or \
                   getattr(config, 'n_layer', None) or \
                   getattr(config, 'num_layers', None)
        
        # Compute d_head
        if n_heads and d_model:
            d_head = d_model // n_heads
            rho = n_heads / d_head
        else:
            d_head = None
            rho = None
        
        # Check normalization type
        norm_type = 'Unknown'
        if hasattr(config, 'layer_norm_eps'):
            norm_type = 'LayerNorm'
        if hasattr(config, 'rms_norm_eps'):
            norm_type = 'RMSNorm'
        
        return {
            'model': model_name,
            'n_layers': n_layers,
            'n_heads': n_heads,
            'd_model': d_model,
            'd_head': d_head,
            'rho': rho,
            'norm_type': norm_type,
            'status': 'OK'
        }
    except Exception as e:
        return {
            'model': model_name,
            'status': f'ERROR: {str(e)[:50]}'
        }

In [ ]:
# Analyze all candidate configs
all_configs = []

print("Scanning model configs...")
print("=" * 80)

for model_name in CANDIDATE_MODELS:
    print(f"  {model_name}...", end=" ")
    result = get_model_rho(model_name)
    all_configs.append(result)
    
    if result['status'] == 'OK':
        print(f"ρ = {result['rho']:.4f}")
    else:
        print(result['status'])

print(f"\nSuccessfully scanned: {sum(1 for c in all_configs if c['status'] == 'OK')} / {len(CANDIDATE_MODELS)}")

In [ ]:
# Filter and sort by ρ
valid_configs = [c for c in all_configs if c['status'] == 'OK' and c['rho'] is not None]
sorted_configs = sorted(valid_configs, key=lambda x: x['rho'], reverse=True)

print("\n" + "=" * 90)
print("MODEL RANKING BY HEAD DENSITY (ρ = n_heads / d_head)")
print("=" * 90)
print(f"\n{'Model':<45} {'Heads':>6} {'d_head':>8} {'ρ':>8} {'Norm':>10} {'Prediction':>12}")
print("-" * 90)

for c in sorted_configs:
    prediction = "DAMPEN?" if c['rho'] >= 0.2 else "EXPAND?"
    marker = "🔵" if c['rho'] >= 0.2 else "🔴"
    
    # Short name
    short_name = c['model'].split('/')[-1]
    
    print(f"{short_name:<45} {c['n_heads']:>6} {c['d_head']:>8} {c['rho']:>8.4f} {c['norm_type']:>10} {marker} {prediction}")

In [ ]:
# Identify HIGH-ρ candidates (ρ ≥ 0.2)
high_rho_candidates = [c for c in sorted_configs if c['rho'] >= 0.2]

print("\n" + "=" * 70)
print("HIGH-ρ CANDIDATES (ρ ≥ 0.2) - EXPECTED TO DAMPEN")
print("=" * 70)

if high_rho_candidates:
    for c in high_rho_candidates:
        print(f"\n  {c['model']}")
        print(f"    ρ = {c['rho']:.4f} (n_heads={c['n_heads']}, d_head={c['d_head']})")
        print(f"    Layers: {c['n_layers']}, Norm: {c['norm_type']}")
else:
    print("\n  ⚠️ No high-ρ candidates found among scanned models!")
    print("  → Pythia may be UNIQUE in its high-ρ configuration")

In [ ]:
# Test prompts
TEST_PROMPTS = [
    "The capital of France is",
    "Water freezes at a temperature of",
    "Actions speak louder than",
    "The quick brown fox jumps over the lazy",
    "In mathematics, the Pythagorean theorem states that",
]

def compute_residual_gain(model, tokenizer, prompts):
    """Compute Residual Stream Gain = ||h_L|| / ||h_{L-1}||."""
    gains = []
    
    for prompt in prompts:
        inputs = tokenizer(prompt, return_tensors="pt")
        if torch.cuda.is_available():
            inputs = {k: v.to(model.device) for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = model(**inputs, output_hidden_states=True)
        
        hidden_states = outputs.hidden_states
        h_L = hidden_states[-1]
        h_Lm1 = hidden_states[-2]
        
        norm_L = torch.norm(h_L[:, -1, :].float(), dim=-1).item()
        norm_Lm1 = torch.norm(h_Lm1[:, -1, :].float(), dim=-1).item()
        
        gain = norm_L / (norm_Lm1 + 1e-10)
        gains.append(gain)
    
    return np.mean(gains), np.std(gains)

In [ ]:
# Select models to test based on GPU memory
# Prioritize high-ρ candidates + reference models

if torch.cuda.is_available():
    available_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"Available GPU memory: {available_mem:.1f} GB")
else:
    available_mem = 8  # Conservative CPU estimate

# Memory estimates (rough)
MEMORY_ESTIMATES = {
    'opt-125m': 0.5, 'opt-350m': 1.5, 'opt-1.3b': 5, 'opt-2.7b': 8, 'opt-6.7b': 15,
    'bloom-560m': 2, 'bloom-1b1': 4, 'bloom-1b7': 6, 'bloom-3b': 10,
    'falcon-7b': 18,
    'gpt-neo-125M': 0.5, 'gpt-neo-1.3B': 5, 'gpt-neo-2.7B': 8,
    'stablelm-base-alpha-3b': 10,
    'pythia-6.9b': 20, 'gpt-j-6B': 18,
}

# Select models that fit
MODELS_TO_TEST = []
for c in sorted_configs:
    short_name = c['model'].split('/')[-1]
    mem_needed = MEMORY_ESTIMATES.get(short_name, 10)
    if mem_needed < (available_mem - 2):
        MODELS_TO_TEST.append(c)

print(f"\nModels to test: {len(MODELS_TO_TEST)}")
for c in MODELS_TO_TEST:
    print(f"  - {c['model'].split('/')[-1]} (ρ = {c['rho']:.4f})")

In [ ]:
# Test each model
results = []

for config in MODELS_TO_TEST:
    model_name = config['model']
    print(f"\n{'='*60}")
    print(f"Testing: {model_name}")
    print(f"ρ = {config['rho']:.4f}")
    print(f"{'='*60}")
    
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            device_map="auto" if torch.cuda.is_available() else None,
            trust_remote_code=True,
            low_cpu_mem_usage=True
        )
        model.eval()
        
        mean_gain, std_gain = compute_residual_gain(model, tokenizer, TEST_PROMPTS)
        
        is_dampening = mean_gain < 1.0
        status = "DAMPENING" if is_dampening else "EXPANSION"
        prediction_correct = (config['rho'] >= 0.2 and is_dampening) or (config['rho'] < 0.2 and not is_dampening)
        
        print(f"\n🔬 RESIDUAL GAIN: {mean_gain:.4f} ± {std_gain:.4f}")
        print(f"   → {status}")
        print(f"   H25 Prediction: {'✅ CORRECT' if prediction_correct else '❌ WRONG'}")
        
        result = config.copy()
        result.update({
            'residual_gain_mean': float(mean_gain),
            'residual_gain_std': float(std_gain),
            'is_dampening': is_dampening,
            'h25_prediction_correct': prediction_correct
        })
        results.append(result)
        
        del model, tokenizer
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            
    except Exception as e:
        print(f"ERROR: {e}")
        result = config.copy()
        result['status'] = f'ERROR: {str(e)[:50]}'
        results.append(result)

print(f"\n\nTested: {len(results)} models")

In [ ]:
# Summary Table
print("\n" + "=" * 100)
print("CROSS-ARCHITECTURE H25 VALIDATION RESULTS")
print("=" * 100)

tested_results = [r for r in results if 'residual_gain_mean' in r]

print(f"\n{'Model':<40} {'ρ':>8} {'Gain':>10} {'Status':>12} {'H25':>8}")
print("-" * 80)

for r in sorted(tested_results, key=lambda x: x['rho'], reverse=True):
    short_name = r['model'].split('/')[-1]
    status = "DAMPEN" if r['is_dampening'] else "EXPAND"
    h25 = "✅" if r['h25_prediction_correct'] else "❌"
    marker = "🔵" if r['is_dampening'] else "🔴"
    
    print(f"{short_name:<40} {r['rho']:>8.4f} {r['residual_gain_mean']:>10.4f} {marker} {status:<10} {h25}")

In [ ]:
# Visualization
if tested_results:
    fig, ax = plt.subplots(figsize=(12, 6))
    
    rhos = [r['rho'] for r in tested_results]
    gains = [r['residual_gain_mean'] for r in tested_results]
    names = [r['model'].split('/')[-1] for r in tested_results]
    colors = ['blue' if g < 1.0 else 'red' for g in gains]
    
    ax.scatter(rhos, gains, c=colors, s=150, zorder=5, edgecolors='black', linewidth=1)
    
    for i, name in enumerate(names):
        ax.annotate(name, (rhos[i], gains[i]), textcoords="offset points", 
                    xytext=(5, 5), fontsize=8, rotation=15)
    
    # Critical lines
    ax.axhline(y=1.0, color='black', linestyle='--', alpha=0.7, label='G=1.0 (Bentov Point)')
    ax.axvline(x=0.2, color='purple', linestyle=':', alpha=0.7, label='ρ=0.2 (Critical?)')
    
    ax.set_xlabel('ρ = n_heads / d_head (Head Density)', fontsize=12)
    ax.set_ylabel('Residual Stream Gain', fontsize=12)
    ax.set_title('H25 Cross-Architecture Validation: ρ vs Residual Gain', fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('high_rho_model_hunt.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\nSaved: high_rho_model_hunt.png")

In [ ]:
# H25 Verdict
print("\n" + "=" * 70)
print("H25 CROSS-ARCHITECTURE VERDICT")
print("=" * 70)

correct_predictions = sum(1 for r in tested_results if r['h25_prediction_correct'])
accuracy = correct_predictions / len(tested_results) if tested_results else 0

print(f"\nH25 Prediction Accuracy: {correct_predictions}/{len(tested_results)} = {accuracy*100:.1f}%")

# Find non-Pythia dampening models
non_pythia_dampen = [r for r in tested_results 
                    if r['is_dampening'] and 'pythia' not in r['model'].lower()]

if non_pythia_dampen:
    print(f"\n✅ FOUND {len(non_pythia_dampen)} NON-PYTHIA DAMPENING MODELS:")
    for r in non_pythia_dampen:
        print(f"   - {r['model'].split('/')[-1]}: ρ = {r['rho']:.4f}, G = {r['residual_gain_mean']:.4f}")
    verdict = "VALIDATED"
else:
    print(f"\n⚠️ No non-Pythia dampening models found")
    print(f"   → Pythia may have UNIQUE properties beyond just high ρ")
    verdict = "PYTHIA_UNIQUE"

In [ ]:
# Save results
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

output_data = {
    'experiment': 'High-ρ Model Hunt',
    'hypothesis': 'H25: ρ ≥ 0.2 → Dampening',
    'date': datetime.now().isoformat(),
    'models_scanned': len(all_configs),
    'models_tested': len(tested_results),
    'prediction_accuracy': accuracy,
    'non_pythia_dampening': [r['model'] for r in non_pythia_dampen] if non_pythia_dampen else [],
    'verdict': verdict,
    'all_configs': all_configs,
    'tested_results': tested_results
}

filename = f'high_rho_model_hunt_{timestamp}.json'
with open(filename, 'w') as f:
    json.dump(output_data, f, indent=2, default=str)

print(f"\nSaved: {filename}")

In [ ]:
# Auto-download
import zipfile

archive_name = f'high_rho_model_hunt_{timestamp}.zip'

with zipfile.ZipFile(archive_name, 'w') as zf:
    zf.write(filename)
    if 'high_rho_model_hunt.png' in globals():
        zf.write('high_rho_model_hunt.png')

print(f"Created archive: {archive_name}")

try:
    from google.colab import files
    files.download(filename)
    files.download('high_rho_model_hunt.png')
    files.download(archive_name)
except ImportError:
    print("Not in Colab - manual download required.")

In [ ]:
# Final Summary
print("\n" + "=" * 70)
print("FINAL SUMMARY: High-ρ Model Hunt")
print("=" * 70)

print(f"\n📊 Models Scanned: {len(all_configs)}")
print(f"🔬 Models Tested: {len(tested_results)}")
print(f"🎯 H25 Accuracy: {accuracy*100:.1f}%")
print(f"\n📋 Verdict: {verdict}")

if non_pythia_dampen:
    print(f"\n✅ Found {len(non_pythia_dampen)} non-Pythia dampening models!")
    print(f"   H25 is ARCHITECTURE-GENERAL, not Pythia-specific.")
else:
    print(f"\n⚠️ Pythia appears to be UNIQUE among tested architectures.")
    print(f"   Dampening may require BOTH high ρ AND other Pythia-specific factors.")